In [253]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
from scipy.optimize import minimize

import random

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm
import sys

In [254]:
%matplotlib widget

%reload_ext autoreload
%autoreload 2

In [255]:
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
sys.path.append('../../../../nadavp/opt_lev_analysis/lib/')
import BeadDataFile
from discharge_tools import *

In [256]:
import bead_util as bu
# import calib_util as cal
import transfer_func_util as tf
import configuration as config

In [235]:
## load all data to a list - step_file_objs
# step_cal_dir = r'/data/old_trap/20220505/bead1/scan1/'

def extract_power_height(dirname):
    max_file = 1000
    power_array = []
    height_array = []
    freq_array = []
    for root, dirnames, filenames in os.walk(dirname):
        filtered_filenames = fnmatch.filter(filenames, '*' + config.extensions['data']) 
        filtered_filenames.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
        for filename in filtered_filenames:
#             print(dirname+filename)
            if '_fpga.h5' in filename[:]:
                try:
                    f = h5py.File(dirname+filename,'r')
                    dat = np.array(f['beads/data/pow_data'])
                    power_array.append(np.mean(dat[::4]))
                    height_array.append(f.attrs['bead_height'])
                    freq_array.append(extract_resonance(dirname+filename[:-8]+'.h5'))
                except:
                    print('***')
    power_array = np.array(power_array)
    height_array = np.array(height_array)
    freq_array = np.array(freq_array)
    
    return height_array, power_array, freq_array

def extract_resonance(filename):
    
    fsamp=5000
    res=50000
    df = bu.DataFile();
    df.load(filename);

    data=df.pos_data
    data_det=signal.detrend(data)
    xpsd, freqs = matplotlib.mlab.psd(data_det[0], Fs = fsamp, NFFT = res)
    psd_tmp = xpsd[100:]
    freq_tmp = freqs[100:]
    return freq_tmp[np.argmax(psd_tmp)]

In [51]:
step_cal_dir = r'/data/old_trap/20220713/bead1/scan2/'
height_array14_, power_array14_, resonances = extract_power_height(step_cal_dir)

In [ ]:
step_cal_dir = r'/data/old_trap/20220509/bead1/scan1/'
height_array2, power_array2, freq_array2 = extract_power_height(step_cal_dir)

In [ ]:
power_array2

In [ ]:
-25000*10/2**15/10*50+50

In [ ]:
420e-12*8/1000*10, 10e-3/3e8, 5/80

In [ ]:
0.069/350*1000*100*2, 1000*0.2/2/2

In [ ]:
np.log10(3.162)

In [338]:
step_cal_dir = r'/data/old_trap/20220713/bead1/scan2/'
height_array14, power_array14, resonances = extract_power_height(step_cal_dir)

## 10um
# step_cal_dir = r'/data/old_trap/20220323/bead1/height_scan1/'
# height_array1, power_array1, resonances = extract_power_height(step_cal_dir)

## 7.5um
# step_cal_dir = r'/data/old_trap/20220727/bead1/scan2/'
# height_array13, power_array13, _ = extract_power_height(step_cal_dir)

In [340]:
power_conversion = -1/222e3*46*0.666
power_conversion2 = 1/484e3*49.2
power_conversion3 = -1/342e3*34
power_conversion4 = -1/36.2e3*5.75

_,ax = plt.subplots(figsize=(9.2,5))
height_array14_guess = np.arange(-2,4,6/len(height_array14))
ax.semilogy(height_array1-2.4, power_array1*power_conversion3, '*', label = '10um')
ax.semilogy(height_array13+6, power_array13*power_conversion4, '.', label = '7.5um')
# ax.semilogy(height_array14+6, power_array14*power_conversion4, '.', label = '7.5um')

# ax.scatter(height_array1, freq_array1, label = 'bead 1 - measurement 1')
ax.legend()
ax.set(ylim=(2,100), ylabel='Power [mW]', xlabel='Height [pixel]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[(2, 100), Text(0, 0.5, 'Power [mW]'), Text(0.5, 0, 'Height [pixel]')]

In [327]:
step_cal_dir = r'/data/old_trap/20220323/bead1/height_scan1/'
height_array1, power_array1, resonances = extract_power_height(step_cal_dir)

_,ax = plt.subplots(figsize=(9.2,5))
ax.scatter(height_array1, power_array1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
power_conversion = -1/222e3*46*0.666
# power_conversion = -1
_,ax = plt.subplots(figsize=(9.2,5))
ax.scatter(reflectance_pixel, reflectance_mw, label = 'bead 1 - reflecatance - after the weekend')

# ax.scatter(height_array1, freq_array1, label = 'bead 1 - measurement 1')
ax.legend()
ax.set( ylabel='Power [mW]', xlabel='Height [$\mu$m]')

In [ ]:
from skimage.registration import phase_cross_correlation as pcc
from skimage.io import imread as imread

import imageio as iio
from skimage import filters
from skimage.color import rgb2gray  # only needed for incorrectly saved images
from skimage.measure import regionprops

h_pcc = []
h_max = []
filename=r'/data/old_trap/20220520/bead1/height/20220520_4/run/image_test_0.bmp'
zeroth_image=imread(filename,as_gray=True)
for i in range(1,117):
    print(i,end=" ")
    filename=r'/data/old_trap/20220520/bead1/height/20220520_4/run/image_test_'+str(i)+'.bmp'
    image=imread(filename,as_gray=True)
    shift,_,_ = pcc(zeroth_image,image,upsample_factor=200)
    h_pcc.append(shift[0])


    threshold_value = filters.threshold_otsu(image)
    labeled_foreground = (image > threshold_value).astype(int)
    properties = regionprops(labeled_foreground, image)
    center_of_mass = properties[0].centroid
    h_max.append(230-center_of_mass[0])

In [ ]:
_,ax = plt.subplots()
ax.scatter(range(116), np.array(h_max)-h_max[0])
ax.scatter(range(116), np.array(h_pcc)-h_pcc[0])

In [242]:
filename = r'/data/old_trap/20220713/bead1/discharge/response_x/X_139Hz_10Vpp_1.h5'
filename2 = r'/data/old_trap/20220713/bead1/discharge/response_y/Y_139Hz_10Vpp_1.h5'
filename3 = r'/data/old_trap/20220713/bead1/discharge/response_z/Z_139Hz_10Vpp_1.h5'

fsamp = 5000
res = 5000

df = bu.DataFile();
df.load(filename);

df2 = bu.DataFile();
df2.load(filename2);

df3 = bu.DataFile();
df3.load(filename3);

data=df.pos_data
data_det=signal.detrend(data)
xpsd, freqs = matplotlib.mlab.psd(data_det[0], Fs = fsamp, NFFT = res)
ypsd, freqs = matplotlib.mlab.psd(data_det[1], Fs = fsamp, NFFT = res)
zpsd, freqs = matplotlib.mlab.psd(data_det[2], Fs = fsamp, NFFT = res)
f = h5py.File(filename[:-3]+'_fpga.h5','r')
height1 = f.attrs['bead_height']

data2=df2.pos_data
data_det2=signal.detrend(data2)
xpsd2, freqs = matplotlib.mlab.psd(data_det2[0], Fs = fsamp, NFFT = res)
ypsd2, freqs = matplotlib.mlab.psd(data_det2[1], Fs = fsamp, NFFT = res)
zpsd2, freqs = matplotlib.mlab.psd(data_det2[2], Fs = fsamp, NFFT = res)
f = h5py.File(filename2[:-3]+'_fpga.h5','r')
height2 = f.attrs['bead_height']

data3=df3.pos_data
data_det3=signal.detrend(data3)
xpsd3, freqs = matplotlib.mlab.psd(data_det3[0], Fs = fsamp, NFFT = res)
ypsd3, freqs = matplotlib.mlab.psd(data_det3[1], Fs = fsamp, NFFT = res)
zpsd3, freqs = matplotlib.mlab.psd(data_det3[2], Fs = fsamp, NFFT = res)
f = h5py.File(filename3[:-3]+'_fpga.h5','r')
height3 = f.attrs['bead_height']

In [241]:
_,ax = plt.subplots()
ax.semilogy(freqs, xpsd, '.-', label='X response',markersize=1,linewidth=0.5)
ax.semilogy(freqs, ypsd2, '.-', label='Y response',markersize=1,linewidth=0.5)
ax.semilogy(freqs, zpsd3, '.-', label='Z response',markersize=1,linewidth=0.5)

ax.legend()
ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel(r'PSD [bits$^2/Hz.$]')
plt.xlim(0.1,400)
#plt.ylim(1e6,1e10)
plt.grid(True)

psd_tmp = xpsd[100:]
freq_tmp = freqs[100:]
freqs[np.argmax(xpsd)], freq_tmp[np.argmax(psd_tmp)]

zx_factor=np.sqrt(zpsd3[freqs==139]/xpsd[freqs==139])
zy_factor=np.sqrt(zpsd3[freqs==139]/ypsd2[freqs==139])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
zx_factor, zy_factor

In [174]:
ll = likelihood_analyser.LikelihoodAnalyser()

## fit parameteres
drive_freq = 139
phase_diff = 1.44
bandwidth = 2
decimate = 10


## extracing the amplitude at a certain frequency 
def get_z_amp(i):
    
    bb = step_file_objs[i]      
    
    ## extract drive phase
    freq2 = np.fft.rfftfreq(len(bb.other_data[1]), d=1./5000)
    fft_angles = np.angle(np.fft.rfft(bb.other_data[1]))
    phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)
#     print(np.std(bb.other_data[1])*100*np.sqrt(2))
    
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2+phase_diff, 
                  'error_A': 0.1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_A': [-100000, 100000], 'fix_phi': True, 'fix_f': True,
                  'print_level': 0}
    
    ## bandpass data
    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    bb.load_other_data()
    xx2 = signal.filtfilt(b, a, bb.pos_data[2])[::decimate]
    ## extract amplitude
    m1_tmp = ll.find_mle_sin(xx2, fsamp=fsamp/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0], m1_tmp.values[2]-phi_tmp2

In [218]:
## load all data to a list - step_file_objs
step_cal_dir = r'/data/old_trap/20220713/bead1/discharge/fine4/'
max_file = 500

step_cal_files = []
for root, dirnames, filenames in os.walk(step_cal_dir):
    for filename in fnmatch.filter(filenames, '*' + config.extensions['data']):
        if '_fpga.h5' in filename:
            continue
        step_cal_files.append(os.path.join(root, filename))
step_cal_files.sort(key = bu.find_str)
num_calib_files = len(step_cal_files)
print(num_calib_files)

# Do the step calibration

step_file_objs = []
for filname in step_cal_files[:]:
    try:
        df = bu.DataFile();
        df.load(filname);
        df.load_other_data()
        step_file_objs.append(df);
    except:
        print('***')

464


In [219]:
## run the function on all calibration files
amps_fine = []
for i in range(num_calib_files):
    print(i,'/',len(step_cal_files))
    amps_fine.append(get_z_amp(i))
amps_fine = np.array(amps_fine)

0 / 464
1 / 464
2 / 464
3 / 464
4 / 464
5 / 464
6 / 464
7 / 464
8 / 464
9 / 464
10 / 464
11 / 464
12 / 464
13 / 464
14 / 464
15 / 464
16 / 464
17 / 464
18 / 464
19 / 464
20 / 464
21 / 464
22 / 464
23 / 464
24 / 464
25 / 464
26 / 464
27 / 464
28 / 464
29 / 464
30 / 464
31 / 464
32 / 464
33 / 464
34 / 464
35 / 464
36 / 464
37 / 464
38 / 464
39 / 464
40 / 464
41 / 464
42 / 464
43 / 464
44 / 464
45 / 464
46 / 464
47 / 464
48 / 464
49 / 464
50 / 464
51 / 464
52 / 464
53 / 464
54 / 464
55 / 464
56 / 464
57 / 464
58 / 464
59 / 464
60 / 464
61 / 464
62 / 464
63 / 464
64 / 464
65 / 464
66 / 464
67 / 464
68 / 464
69 / 464
70 / 464
71 / 464
72 / 464
73 / 464
74 / 464
75 / 464
76 / 464
77 / 464
78 / 464
79 / 464
80 / 464
81 / 464
82 / 464
83 / 464
84 / 464
85 / 464
86 / 464
87 / 464
88 / 464
89 / 464
90 / 464
91 / 464
92 / 464
93 / 464
94 / 464
95 / 464
96 / 464
97 / 464
98 / 464
99 / 464
100 / 464
101 / 464
102 / 464
103 / 464
104 / 464
105 / 464
106 / 464
107 / 464
108 / 464
109 / 464
110 / 464


/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [215]:
scaleZ

-47.080894030421405

In [221]:
amps_plot = np.array(amps_fine.T[0,:])
# scaleZ = np.mean(amps_plot[51:55])/2
amps_plot /= scaleZ

_,ax = plt.subplots(figsize=(9.5,4))
ax.scatter(range(len(amps_plot)), amps_plot, label='fit phase')

# ax.set(xlabel='dataset', ylabel='electron charges', ylim=(-1, 14.5), xlim=(12,None))
ax.grid(axis='y')
ax.minorticks_on()
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
from matplotlib.ticker import MaxNLocator

ax.yaxis.set_major_locator(MaxNLocator(integer=True))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
force_sens = np.std(amps_plot[-6:])*50/4e-3*0.66*1.6e-19*np.sqrt(10)
print("estimated force sensitivity in one second [N]: ", force_sens)

In [ ]:
factor_z = 1*50/4e-3*0.66*1.6e-19/scaleZ

In [178]:
amps_phase = np.array(amps_fine.T[1,:])

_,ax = plt.subplots(figsize=(9.5,4))
ax.scatter(range(len(amps_phase)), amps_phase, label='fit phase')

ax.set(xlabel='dataset', ylabel='phase', xlim=(None,None))
ax.grid(axis='y')
ax.minorticks_on()
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
from matplotlib.ticker import MaxNLocator

ax.yaxis.set_major_locator(MaxNLocator(integer=True))

np.mean(amps_phase[:80])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([ 1.44])

In [ ]:
factor_z

In [252]:
filename = r'/data/old_trap/20220603/bead1/discharge/fine/Z_139Hz_50Vpp_100.h5'
filename = r'/data/old_trap/20220715/bead1/scan1/5mbar_zcool_10.h5'

fsamp = 5000
res = 5000

df = bu.DataFile();
df.load(filename);

data=df.pos_data
data_det=signal.detrend(data)
xpsd, freqs = matplotlib.mlab.psd(data_det[0], Fs = fsamp, NFFT = res)
ypsd, freqs = matplotlib.mlab.psd(data_det[1], Fs = fsamp, NFFT = res)
zpsd, freqs = matplotlib.mlab.psd(data_det[2], Fs = fsamp, NFFT = res)

_,ax = plt.subplots()
ax.loglog(freqs, np.sqrt(xpsd), '.-', label="X",markersize=1,linewidth=0.5)
ax.loglog(freqs, np.sqrt(ypsd), '.-', label="Y",markersize=1,linewidth=0.5)
# ax.loglog(freqs, np.sqrt(zpsd), '.-', label="Z",markersize=1,linewidth=0.5)
f = h5py.File(filename[:-3]+'_fpga.h5','r')
height_tmp = f.attrs['bead_height']
print('height: ', height_tmp)
ax.legend()
ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel(r'ASD [N$/\sqrt{Hz}$]')
# plt.xlim(0.1,400)
#plt.ylim(1e6,1e10)
plt.grid(True)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

height:  5.19
